In [78]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [79]:
monday= pd.read_csv('data/monday.csv', sep=';')
monday['timestamp']=pd.to_datetime(monday['timestamp'])
monday.set_index('timestamp', inplace=True)

In [80]:
df = monday.groupby('customer_no').resample('T').ffill()
df.drop('customer_no', axis=1, inplace=True)
df


location
customer_no timestamp                    
1           2019-09-02 07:03:00     dairy
            2019-09-02 07:04:00     dairy
            2019-09-02 07:05:00  checkout
2           2019-09-02 07:03:00     dairy
            2019-09-02 07:04:00     dairy
...                                   ...
1444        2019-09-02 21:48:00    spices
            2019-09-02 21:49:00  checkout
1445        2019-09-02 21:49:00     dairy
1446        2019-09-02 21:50:00     dairy
1447        2019-09-02 21:50:00     fruit

[10765 rows x 1 columns]

In [81]:
df = df.reset_index().set_index('timestamp')
df


,customer_no,location
timestamp,,
2019-09-02 07:03:00,1,dairy
2019-09-02 07:04:00,1,dairy
2019-09-02 07:05:00,1,checkout
2019-09-02 07:03:00,2,dairy
2019-09-02 07:04:00,2,dairy
...,...,...
2019-09-02 21:48:00,1444,spices
2019-09-02 21:49:00,1444,checkout
2019-09-02 21:49:00,1445,dairy


In [82]:
df['previous'] = df['location'].shift(1)
df

,customer_no,location,previous
timestamp,,,
2019-09-02 07:03:00,1,dairy,NaN
2019-09-02 07:04:00,1,dairy,dairy
2019-09-02 07:05:00,1,checkout,dairy
2019-09-02 07:03:00,2,dairy,checkout
2019-09-02 07:04:00,2,dairy,dairy
...,...,...,...
2019-09-02 21:48:00,1444,spices,dairy
2019-09-02 21:49:00,1444,checkout,spices
2019-09-02 21:49:00,1445,dairy,checkout


In [98]:
#df.loc[df['previous']=='checkout']['previous'].replace('entrance', inplace=True)
#df
df['previous'].replace({'checkout':'entrance'}, inplace=True)
df['previous'].fillna('entrance', inplace=True)
df
#df.drop('customer_no', inplace=True)
#df.drop('index', axis=1, inplace=True)

,customer_no,location,previous
timestamp,,,
2019-09-02 07:03:00,1,dairy,entrance
2019-09-02 07:04:00,1,dairy,dairy
2019-09-02 07:05:00,1,checkout,dairy
2019-09-02 07:03:00,2,dairy,entrance
2019-09-02 07:04:00,2,dairy,dairy
...,...,...,...
2019-09-02 21:48:00,1444,spices,dairy
2019-09-02 21:49:00,1444,checkout,spices
2019-09-02 21:49:00,1445,dairy,entrance


In [99]:
df.groupby(['location', 'previous']).count().unstack()

customer_no                                
previous       dairy  drinks entrance   fruit spices
location                                            
checkout       310.0   426.0      NaN   524.0  177.0
dairy         2601.0    23.0    410.0   225.0  236.0
drinks         218.0  1250.0    230.0   132.0  217.0
fruit          181.0   186.0    517.0  1554.0  120.0
spices         185.0   162.0    281.0   122.0  478.0

In [100]:
P= pd.crosstab(df['previous'], df['location'], normalize=0) # <-- 0 axis
P

location,checkout,dairy,drinks,fruit,spices
previous,,,,,
dairy,0.088698,0.744206,0.062375,0.051788,0.052933
drinks,0.208109,0.011236,0.610650,0.090865,0.079140
entrance,0.000000,0.285118,0.159944,0.359527,0.195410
fruit,0.204928,0.087994,0.051623,0.607743,0.047712
spices,0.144137,0.192182,0.176710,0.097720,0.389251


In [101]:
P = P[['dairy','drinks','fruit','spices','checkout']]
P

location,dairy,drinks,fruit,spices,checkout
previous,,,,,
dairy,0.744206,0.062375,0.051788,0.052933,0.088698
drinks,0.011236,0.610650,0.090865,0.079140,0.208109
entrance,0.285118,0.159944,0.359527,0.195410,0.000000
fruit,0.087994,0.051623,0.607743,0.047712,0.204928
spices,0.192182,0.176710,0.097720,0.389251,0.144137


In [102]:
P.index

Index(['dairy', 'drinks', 'entrance', 'fruit', 'spices'], dtype='object', name='previous')

In [104]:
P=P.reindex(['entrance', 'dairy', 'drinks', 'fruit', 'spices'])
P=P.drop('entrance', axis=0)
P


location,dairy,drinks,fruit,spices,checkout
previous,,,,,
dairy,0.744206,0.062375,0.051788,0.052933,0.088698
drinks,0.011236,0.610650,0.090865,0.079140,0.208109
fruit,0.087994,0.051623,0.607743,0.047712,0.204928
spices,0.192182,0.176710,0.097720,0.389251,0.144137


In [105]:
new_row = pd.Series(data={'dairy':0, 'drinks':0, 'fruit':0, 'spices':0, 'checkout':1}, name='checkout')
#append row to the dataframe
P = P.append(new_row, ignore_index=False)
P


location,dairy,drinks,fruit,spices,checkout
previous,,,,,
dairy,0.744206,0.062375,0.051788,0.052933,0.088698
drinks,0.011236,0.610650,0.090865,0.079140,0.208109
fruit,0.087994,0.051623,0.607743,0.047712,0.204928
spices,0.192182,0.176710,0.097720,0.389251,0.144137
checkout,0.000000,0.000000,0.000000,0.000000,1.000000


In [106]:
customer = np.array([0,0,1,0,0])

In [118]:
customer.dot(P).dot(P).dot(P).dot(P).dot(P).dot(P).dot(P).dot(P).dot(P).dot(P).dot(P).dot(P).dot(P).dot(P).dot(P).dot(P).dot(P).dot(P).dot(P).dot(P).dot(P).dot(P).dot(P).dot(P).dot(P).dot(P).dot(P).dot(P)


array([3.56661311e-03, 2.05493161e-03, 1.92202699e-03, 9.61920774e-04,
       9.91494508e-01])